In [2]:
import requests
import json
from math import ceil

## Get the articles

In [3]:
openalex_url = "https://api.openalex.org/works"

In [6]:
def download_ai_ml_articles(url):
    params = {
        #"filter": "concepts.id:('ai_concept_id','ml_concept_id')"  # Replace with actual concept IDs.
        #"type": "article"
    }
    
    articles = []
    
    page = 1
    
    while True:
        params['page'] = page
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            print(data['results'][0]['keywords'])
            print(data['results'][0]['concepts'])
            articles.extend(data['results'])


            if 'next' not in data or not data['next']:
                break

            page += 1
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break
        break

In [7]:
download_ai_ml_articles(openalex_url)

[{'keyword': 'folin phenol reagent', 'score': 0.8403}, {'keyword': 'protein', 'score': 0.4743}]
[{'id': 'https://openalex.org/C40875361', 'wikidata': 'https://www.wikidata.org/wiki/Q2356542', 'display_name': 'Reagent', 'level': 2, 'score': 0.97526133}, {'id': 'https://openalex.org/C185592680', 'wikidata': 'https://www.wikidata.org/wiki/Q2329', 'display_name': 'Chemistry', 'level': 0, 'score': 0.83260155}, {'id': 'https://openalex.org/C2777702071', 'wikidata': 'https://www.wikidata.org/wiki/Q130336', 'display_name': 'Phenol', 'level': 2, 'score': 0.5855713}, {'id': 'https://openalex.org/C43617362', 'wikidata': 'https://www.wikidata.org/wiki/Q170050', 'display_name': 'Chromatography', 'level': 1, 'score': 0.55636495}, {'id': 'https://openalex.org/C178790620', 'wikidata': 'https://www.wikidata.org/wiki/Q11351', 'display_name': 'Organic chemistry', 'level': 1, 'score': 0.2482543}]


Too many of them, and I figured out that there is a **concepts** collection which already contains the counts that I eed

## Get the concepts

In [8]:
concepts_url = "https://api.openalex.org/concepts"

In [9]:
metadata = {
    "count": 65073,
    "db_response_time_ms": 48,
    "page": 1,
    "per_page": 25,
    "groups_count": None
}
pages = ceil(metadata['count']/metadata['per_page'])
pages

2603

In [36]:
def download_concepts(url):
    params = {}
    
    concepts = []
    concept_count = 0
    
    page = 1
    with open('concepts_per_page/concepts.jsonl', 'a+') as file:
        for page in range(1, pages+1):
            print(f'Fetching page: {page}')
            
            params['page'] = page
            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                page_results = data['results']
                file.write(json.dumps(page_results) + '\n')
                
                for concept in page_results:
                    print(f"Building up concept: {concept['display_name']}")
                    concept_json = {
                        "name": concept['display_name'],
                        "ancestors": [],
                        "related_concepts": [],
                        "counts_by_year": concept['counts_by_year']
                    }
                    for ancestor in concept['ancestors']:
                        concept_json['ancestors'].append(ancestor['display_name'])
                    for related_concept in concept['related_concepts']:
                        concept_json['related_concepts'].append(related_concept['display_name'])
                
                    with open('openalex_concepts.jsonl', 'a+') as f:  
                        f.write(json.dumps(concept_json) + '\n')

                page += 1
            else:
                print(f"Failed to fetch data: {response.status_code}")
                break
            break

In [37]:
download_concepts(concepts_url)

Fetching page: 1
Building up concept: Computer science
Building up concept: Medicine
Building up concept: Biology
Building up concept: Physics
Building up concept: Political science
Building up concept: Chemistry
Building up concept: Philosophy
Building up concept: Engineering
Building up concept: Mathematics
Building up concept: Psychology
Building up concept: Internal medicine
Building up concept: Materials science
Building up concept: Art
Building up concept: Geography
Building up concept: Law
Building up concept: Business
Building up concept: Sociology
Building up concept: Humanities
Building up concept: Economics
Building up concept: Quantum mechanics
Building up concept: Biochemistry
Building up concept: Organic chemistry
Building up concept: Artificial intelligence
Building up concept: Geology
Building up concept: History


In [10]:
#https://api.openalex.org/works?filter=concepts.id:https://openalex.org/C41008148
